In [28]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import RobustScaler
from torch.nn.utils.rnn import pad_sequence

class CryptoDataset(Dataset):
    def __init__(self, features, labels, augment=True):
        self.features = features
        self.labels = labels
        self.augment = augment
        
        # Use RobustScaler instead of StandardScaler for better handling of outliers
        self.scaler = RobustScaler(quantile_range=(25.0, 75.0))
        
        # Clip extreme values before scaling
        max_val = np.percentile(np.concatenate([f.flatten() for f in features]), 99)
        min_val = np.percentile(np.concatenate([f.flatten() for f in features]), 1)
        
        # Clip and scale features
        self.scaled_features = []
        for feature in features:
            # Clip extreme values
            clipped = np.clip(feature, min_val, max_val)
            
            # Handle any remaining infinities or NaNs
            clipped = np.nan_to_num(clipped, nan=0.0, posinf=max_val, neginf=min_val)
            
            # Reshape for scaling
            orig_shape = clipped.shape
            flat = clipped.reshape(clipped.shape[0], -1)
            
            # Fit and transform
            if len(self.scaled_features) == 0:  # Only fit on first batch
                scaled = self.scaler.fit_transform(flat)
            else:
                scaled = self.scaler.transform(flat)
            
            # Reshape back and add to scaled features
            self.scaled_features.append(scaled.reshape(orig_shape))
    
    def __len__(self):
        return len(self.features)
    
    def augment_feature(self, feature):
        # Add random noise
        noise = np.random.normal(0, 0.01, feature.shape)
        augmented = feature + noise
        
        # Random scaling
        scale = np.random.uniform(0.95, 1.05)
        augmented *= scale
        
        return augmented
    
    def __getitem__(self, idx):
        feature_matrix = self.scaled_features[idx]
        
        # Apply augmentation during training
        if self.augment:
            feature_matrix = self.augment_feature(feature_matrix)
        
        feature_matrix = torch.tensor(feature_matrix, dtype=torch.float32)
        label = self.labels[idx]
        
        # Safety check
        if torch.isnan(feature_matrix).any() or torch.isinf(feature_matrix).any():
            feature_matrix = torch.nan_to_num(feature_matrix, nan=0.0, posinf=1.0, neginf=-1.0)
        
        return feature_matrix, label

def collate_fn(batch):
    """Custom collate function with additional safety checks"""
    # Separate features and labels
    features = []
    labels = []
    
    for item in batch:
        feature, label = item
        
        # Additional safety check
        if torch.isnan(feature).any() or torch.isinf(feature).any():
            continue
        
        features.append(feature)
        labels.append(label)
    
    if not features:  # If all features were invalid
        raise ValueError("All features in batch contained NaN or Inf values")
    
    # Pad features to the maximum length in the batch
    padded_features = pad_sequence(features, batch_first=True)
    
    # Convert labels to tensor
    labels = torch.stack([torch.tensor(l) for l in labels])
    
    return padded_features, labels

def load_and_preprocess_data(directory):
    """Load and preprocess the cryptographic feature matrices"""
    feature_data = []
    labels = []
    algo_to_label = {'3desenc': 0, 'AES': 1, 'el_gamal': 2, 'kasumi': 3, 'rsa': 4}
    
    print("Loading data from directories...")
    for algo, label in algo_to_label.items():
        algo_dir = os.path.join(directory, algo)
        if not os.path.exists(algo_dir):                                           
            raise FileNotFoundError(f"Directory not found: {algo_dir}")
            
        files = [f for f in os.listdir(algo_dir) if f.endswith('_features.txt')]
        print(f"Found {len(files)} files for algorithm {algo}")                         
        
        for file in files:
            file_path = os.path.join(algo_dir, file)
            try:
                features = np.loadtxt(file_path)
                
                # Handle 1D arrays
                if len(features.shape) == 1:
                    features = features.reshape(1, -1)
                
                # Check for NaN or Inf values
                if np.isnan(features).any() or np.isinf(features).any():
                    print(f"Warning: Found NaN or Inf in {file_path}, cleaning...")
                    features = np.nan_to_num(features, nan=0.0, posinf=1.0, neginf=-1.0)
                
                feature_data.append(features)
                labels.append(label)
                
            except Exception as e:
                print(f"Error loading file {file_path}: {str(e)}")
                continue
    
    print(f"\nTotal samples loaded: {len(feature_data)}")
    print(f"Feature matrix shapes: {[f.shape for f in feature_data[:5]]} ...")
    
    # Convert labels to tensor
    labels = torch.tensor(labels, dtype=torch.long)
    
    return feature_data, labels 

In [29]:
    # Load and preprocess data
data_dir = "/home/21uec131/crptonet"  # Update this path
features, labels = load_and_preprocess_data(data_dir)
    
    

Loading data from directories...
Found 8805 files for algorithm 3desenc
Found 8805 files for algorithm AES
Found 8805 files for algorithm el_gamal
Found 8805 files for algorithm kasumi
Found 8805 files for algorithm rsa

Total samples loaded: 44025
Feature matrix shapes: [(1240, 21), (111, 21), (165, 21), (98, 21), (66, 21)] ...


In [30]:
def print_dataset_statistics(features, labels, batch_size, train_size, val_size, test_size):
    """Print comprehensive dataset and training statistics"""
    print("\n" + "="*50)
    print("DATASET AND TRAINING STATISTICS")
    print("="*50)
    
    # Dataset Statistics
    print("\n1. DATASET OVERVIEW")
    print("-"*20)
    print(f"Total samples: {len(features)}")
    print(f"Number of classes: {len(set(labels.numpy()))}")
    print(f"Feature matrix shapes: {[f.shape for f in features[:5]]} ...")
    
    # Data Split Statistics
    print("\n2. DATA SPLITTING")
    print("-"*20)
    print(f"Training samples: {train_size} ({train_size/len(features):.1%})")
    print(f"Validation samples: {val_size} ({val_size/len(features):.1%})")
    print(f"Testing samples: {test_size} ({test_size/len(features):.1%})")
    
    # Training Parameters
    print("\n3. TRAINING PARAMETERS")
    print("-"*20)
    print(f"Batch size: {batch_size}")
    print(f"Number of training batches: {train_size // batch_size}")
    print(f"Number of validation batches: {val_size // batch_size}")
    print(f"Number of test batches: {test_size // batch_size}")
    
    # Class Distribution
    print("\n4. CLASS DISTRIBUTION")
    print("-"*20)
    unique, counts = np.unique(labels, return_counts=True)
    for label, count in zip(unique, counts):
        print(f"Class {label}: {count} samples ({count/len(labels):.1%})")
    
    # Hardware Configuration
    print("\n5. HARDWARE CONFIGURATION")
    print("-"*20)
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    print(f"Device being used: {'cuda' if torch.cuda.is_available() else 'cpu'}")
    if torch.cuda.is_available():
        print(f"GPU Model: {torch.cuda.get_device_name(0)}")
    
    # Model Parameters
    print("\n6. MODEL CONFIGURATION")
    print("-"*20)
    print(f"Input channels: {features[0].shape[1]}")
    print(f"Learning rate: 0.0001")
    print(f"Weight decay: 0.01")
    print(f"Number of workers: 8")
    print(f"Early stopping patience: 15")
    
    print("\n" + "="*50 + "\n")



In [31]:
dataset = CryptoDataset(features, labels)
print_dataset_statistics (
        features=features,
        labels=labels,
        batch_size=64,
        train_size=int(0.7 * len(dataset)),
        val_size=int(0.15 * len(dataset)),
        test_size=len(dataset) - int(0.7 * len(dataset)) - int(0.15 * len(dataset))
)


DATASET AND TRAINING STATISTICS

1. DATASET OVERVIEW
--------------------
Total samples: 44025
Number of classes: 5
Feature matrix shapes: [(1240, 21), (111, 21), (165, 21), (98, 21), (66, 21)] ...

2. DATA SPLITTING
--------------------
Training samples: 30817 (70.0%)
Validation samples: 6603 (15.0%)
Testing samples: 6605 (15.0%)

3. TRAINING PARAMETERS
--------------------
Batch size: 64
Number of training batches: 481
Number of validation batches: 103
Number of test batches: 103

4. CLASS DISTRIBUTION
--------------------
Class 0: 8805 samples (20.0%)
Class 1: 8805 samples (20.0%)
Class 2: 8805 samples (20.0%)
Class 3: 8805 samples (20.0%)
Class 4: 8805 samples (20.0%)

5. HARDWARE CONFIGURATION
--------------------
Number of GPUs available: 8
Device being used: cuda
GPU Model: Tesla V100-SXM2-32GB

6. MODEL CONFIGURATION
--------------------
Input channels: 21
Learning rate: 0.0001
Weight decay: 0.01
Number of workers: 8
Early stopping patience: 15




In [32]:
import numpy as np
import torch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import MiniBatchKMeans, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt
from tqdm import tqdm

class FeatureAnalyzer:
    def __init__(self, features, labels, n_components=2, batch_size=1000):
        self.features = features
        self.labels = labels
        self.n_components = n_components
        self.batch_size = batch_size
        self.flattened_features = None
    
    def preprocess_features(self):
        """Memory-efficient feature preprocessing"""
        print("Preprocessing features...")
        
        # Process features in batches
        processed_features = []
        for i in tqdm(range(0, len(self.features), self.batch_size), desc="Processing batches"):
            batch = self.features[i:i + self.batch_size]
            
            # Process each feature in the batch
            batch_processed = []
            for feature in batch:
                # Flatten and compute feature statistics
                flat = feature.flatten()
                feature_mean = np.mean(flat)
                feature_std = np.std(flat) + 1e-8
                
                # Normalize the feature
                normalized = (flat - feature_mean) / feature_std
                batch_processed.append(normalized)
            
            # Stack batch results
            processed_features.extend(batch_processed)
            
            # Clear memory
            del batch
            del batch_processed
        
        # Find minimum length
        min_len = min(len(f) for f in processed_features)
        
        # Truncate all features to minimum length
        self.flattened_features = np.vstack([f[:min_len] for f in processed_features])
        print(f"Preprocessed feature shape: {self.flattened_features.shape}")
        
        return self.flattened_features
    
    def run_pca(self):
        """Memory-efficient PCA"""
        print("\nPerforming PCA analysis...")
        try:
            # Use incremental PCA for memory efficiency
            pca = PCA(n_components=self.n_components, svd_solver='randomized')
            pca_result = pca.fit_transform(self.flattened_features)
            
            explained_variance = pca.explained_variance_ratio_
            
            # Plot explained variance
            plt.figure(figsize=(10, 5))
            plt.plot(range(1, len(explained_variance) + 1), 
                    np.cumsum(explained_variance), 'bo-')
            plt.xlabel('Number of Components')
            plt.ylabel('Cumulative Explained Variance Ratio')
            plt.title('PCA Explained Variance')
            plt.savefig('pca_variance.png')
            plt.close()
            
            return pca_result, explained_variance
        except Exception as e:
            print(f"Error in PCA: {str(e)}")
            return None, None
    
    def run_tsne(self):
        """Memory-efficient t-SNE with consistent sampling"""
        print("\nPerforming t-SNE analysis...")
        try:
            # Sample data if too large
            self.max_samples = 10000  # Make this a class variable
            if len(self.flattened_features) > self.max_samples:
                self.sample_indices = np.random.choice(
                    len(self.flattened_features), 
                    self.max_samples, 
                    replace=False
                )
                sample_features = self.flattened_features[self.sample_indices]
                sample_labels = self.labels[self.sample_indices]
                self.is_sampled = True
            else:
                sample_features = self.flattened_features
                sample_labels = self.labels
                self.is_sampled = False
            
            tsne = TSNE(n_components=self.n_components, random_state=42)
            tsne_result = tsne.fit_transform(sample_features)
            return tsne_result, sample_labels
        except Exception as e:
            print(f"Error in t-SNE: {str(e)}")
            return None, None
    
    def run_clustering(self):
        """Memory-efficient clustering with consistent sampling"""
        print("\nPerforming clustering analysis...")
        try:
            # Use same sampling as t-SNE if needed
            if hasattr(self, 'is_sampled') and self.is_sampled:
                sample_features = self.flattened_features[self.sample_indices]
            else:
                sample_features = self.flattened_features
            
            # Use MiniBatchKMeans
            kmeans = MiniBatchKMeans(
                n_clusters=5, 
                batch_size=1000,
                random_state=42
            )
            kmeans_labels = kmeans.fit_predict(sample_features)
            
            # DBSCAN on sampled data
            dbscan = DBSCAN(eps=0.5, min_samples=5)
            dbscan_labels = dbscan.fit_predict(sample_features)
            
            # Calculate metrics
            metrics = {
                'kmeans_silhouette': silhouette_score(
                    sample_features, 
                    kmeans_labels
                ),
                'kmeans_calinski': calinski_harabasz_score(
                    sample_features, 
                    kmeans_labels
                )
            }
            
            if len(np.unique(dbscan_labels)) > 1:
                metrics.update({
                    'dbscan_silhouette': silhouette_score(
                        sample_features, 
                        dbscan_labels
                    ),
                    'dbscan_calinski': calinski_harabasz_score(
                        sample_features, 
                        dbscan_labels
                    )
                })
            
            return kmeans_labels, dbscan_labels, metrics
        except Exception as e:
            print(f"Error in clustering: {str(e)}")
            return None, None, {}
    
    def plot_results(self, pca_result, tsne_result, kmeans_labels, dbscan_labels):
        """Plot all analysis results with consistent sampling"""
        if any(x is None for x in [pca_result, tsne_result, kmeans_labels, dbscan_labels]):
            print("Skipping plotting due to missing results")
            return
            
        print("\nPlotting results...")
        try:
            fig, axes = plt.subplots(2, 2, figsize=(20, 20))
            
            # Use sampled data consistently
            if hasattr(self, 'is_sampled') and self.is_sampled:
                plot_features = pca_result[self.sample_indices]
                plot_labels = self.labels[self.sample_indices]
            else:
                plot_features = pca_result
                plot_labels = self.labels
            
            # Plot PCA with true labels
            self._scatter_plot(
                axes[0, 0], 
                plot_features, 
                plot_labels,
                'PCA Results with True Labels', 
                'PC1', 
                'PC2'
            )
            
            # Plot t-SNE with true labels
            self._scatter_plot(
                axes[0, 1], 
                tsne_result, 
                plot_labels,
                't-SNE Results with True Labels', 
                't-SNE1', 
                't-SNE2'
            )
            
            # Plot K-means clustering
            self._scatter_plot(
                axes[1, 0], 
                plot_features, 
                kmeans_labels,
                'PCA Results with K-means Clustering', 
                'PC1', 
                'PC2'
            )
            
            # Plot DBSCAN clustering
            self._scatter_plot(
                axes[1, 1], 
                plot_features, 
                dbscan_labels,
                'PCA Results with DBSCAN Clustering', 
                'PC1', 
                'PC2'
            )
            
            plt.tight_layout()
            plt.savefig('feature_analysis.png')
            plt.close()
        except Exception as e:
            print(f"Error in plotting: {str(e)}")
    
    def _scatter_plot(self, ax, data, labels, title, xlabel, ylabel):
        """Helper function to create scatter plots"""
        scatter = ax.scatter(data[:, 0], data[:, 1], c=labels, cmap='tab10')
        ax.set_title(title)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        plt.colorbar(scatter, ax=ax)
    
    def analyze_features(self):
        """Run complete feature analysis with consistent sampling"""
        # Preprocess features
        self.preprocess_features()
        
        # Run analyses
        pca_result, explained_variance = self.run_pca()
        tsne_result, sampled_labels = self.run_tsne()
        kmeans_labels, dbscan_labels, clustering_metrics = self.run_clustering()
        
        # Plot results
        self.plot_results(pca_result, tsne_result, kmeans_labels, dbscan_labels)
        
        # Return results
        results = {
            'pca_result': pca_result,
            'tsne_result': tsne_result,
            'explained_variance': explained_variance if explained_variance is not None else np.array([0]),
            'clustering_metrics': clustering_metrics
        }
        
        # Print analysis summary
        if explained_variance is not None:
            print("\nAnalysis Summary:")
            print(f"Total variance explained by {self.n_components} PCA components: {sum(explained_variance):.2%}")
            print("\nClustering Metrics:")
            for metric, value in clustering_metrics.items():
                print(f"{metric}: {value:.3f}")
        
        return results

def run_feature_analysis(features, labels):
    """Main function to run feature analysis"""
    analyzer = FeatureAnalyzer(features, labels, batch_size=1000)
    results = analyzer.analyze_features()
    return results 

In [33]:
# Add feature analysis before training
print("\nRunning unsupervised feature analysis...")
analysis_results = run_feature_analysis(features, labels)
    
# Print feature analysis insights
print("\nFeature Analysis Insights:")
print(f"PCA Explained Variance: {analysis_results['explained_variance'].sum():.2%}")
print("\nClustering Metrics:")
for metric, value in analysis_results['clustering_metrics'].items():
    print(f"{metric}: {value:.3f}")
    
# Continue with model training if features look good
if analysis_results['explained_variance'].sum() > 0.7:  # 70% variance explained
    print("\nFeatures look good, proceeding with training...")


Running unsupervised feature analysis...
Preprocessing features...


Processing batches: 100%|██████████| 45/45 [00:11<00:00,  3.82it/s]


Preprocessed feature shape: (44025, 126)

Performing PCA analysis...

Performing t-SNE analysis...

Performing clustering analysis...


/home/21uec131/miniconda3/envs/crptonet/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)



Plotting results...

Analysis Summary:
Total variance explained by 2 PCA components: 20.42%

Clustering Metrics:
kmeans_silhouette: 0.089
kmeans_calinski: 669.733
dbscan_silhouette: -0.190
dbscan_calinski: 20.184

Feature Analysis Insights:
PCA Explained Variance: 20.42%

Clustering Metrics:
kmeans_silhouette: 0.089
kmeans_calinski: 669.733
dbscan_silhouette: -0.190
dbscan_calinski: 20.184


In [34]:
import numpy as np
import torch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import MiniBatchKMeans, DBSCAN
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt
from tqdm import tqdm

class FeatureAnalyzer:
    def __init__(self, features, labels, n_components=2, batch_size=1000):
        self.features = features
        self.labels = labels
        self.n_components = n_components
        self.batch_size = batch_size
        self.flattened_features = None
    
    def preprocess_features(self):
        """Memory-efficient feature preprocessing"""
        print("Preprocessing features...")
        
        # Process features in batches
        processed_features = []
        for i in tqdm(range(0, len(self.features), self.batch_size), desc="Processing batches"):
            batch = self.features[i:i + self.batch_size]
            
            # Process each feature in the batch
            batch_processed = []
            for feature in batch:
                # Flatten and compute feature statistics
                flat = feature.flatten()
                feature_mean = np.mean(flat)
                feature_std = np.std(flat) + 1e-8
                
                # Normalize the feature
                normalized = (flat - feature_mean) / feature_std
                batch_processed.append(normalized)
            
            # Stack batch results
            processed_features.extend(batch_processed)
            
            # Clear memory
            del batch
            del batch_processed
        
        # Find minimum length
        min_len = min(len(f) for f in processed_features)
        
        # Truncate all features to minimum length
        self.flattened_features = np.vstack([f[:min_len] for f in processed_features])
        print(f"Preprocessed feature shape: {self.flattened_features.shape}")
        
        return self.flattened_features
    
    def run_pca(self):
        """Memory-efficient PCA"""
        print("\nPerforming PCA analysis...")
        try:
            # Use incremental PCA for memory efficiency
            pca = PCA(n_components=self.n_components, svd_solver='randomized')
            pca_result = pca.fit_transform(self.flattened_features)
            
            explained_variance = pca.explained_variance_ratio_
            
            # Plot explained variance
            plt.figure(figsize=(10, 5))
            plt.plot(range(1, len(explained_variance) + 1), 
                    np.cumsum(explained_variance), 'bo-')
            plt.xlabel('Number of Components')
            plt.ylabel('Cumulative Explained Variance Ratio')
            plt.title('PCA Explained Variance')
            plt.savefig('pca_variance.png')
            plt.close()
            
            return pca_result, explained_variance
        except Exception as e:
            print(f"Error in PCA: {str(e)}")
            return None, None
    
    def run_tsne(self):
        """Memory-efficient t-SNE with consistent sampling"""
        print("\nPerforming t-SNE analysis...")
        try:
            # Sample data if too large
            self.max_samples = 10000  # Make this a class variable
            if len(self.flattened_features) > self.max_samples:
                self.sample_indices = np.random.choice(
                    len(self.flattened_features), 
                    self.max_samples, 
                    replace=False
                )
                sample_features = self.flattened_features[self.sample_indices]
                sample_labels = self.labels[self.sample_indices]
                self.is_sampled = True
            else:
                sample_features = self.flattened_features
                sample_labels = self.labels
                self.is_sampled = False
            
            tsne = TSNE(n_components=self.n_components, random_state=42)
            tsne_result = tsne.fit_transform(sample_features)
            return tsne_result, sample_labels
        except Exception as e:
            print(f"Error in t-SNE: {str(e)}")
            return None, None
    
    def run_clustering(self):
        """Memory-efficient clustering with consistent sampling"""
        print("\nPerforming clustering analysis...")
        try:
            # Use same sampling as t-SNE if needed
            if hasattr(self, 'is_sampled') and self.is_sampled:
                sample_features = self.flattened_features[self.sample_indices]
            else:
                sample_features = self.flattened_features
            
            # Use MiniBatchKMeans
            kmeans = MiniBatchKMeans(
                n_clusters=5, 
                batch_size=1000,
                random_state=42
            )
            kmeans_labels = kmeans.fit_predict(sample_features)
            
            # DBSCAN on sampled data
            dbscan = DBSCAN(eps=0.5, min_samples=5)
            dbscan_labels = dbscan.fit_predict(sample_features)
            
            # Calculate metrics
            metrics = {
                'kmeans_silhouette': silhouette_score(
                    sample_features, 
                    kmeans_labels
                ),
                'kmeans_calinski': calinski_harabasz_score(
                    sample_features, 
                    kmeans_labels
                )
            }
            
            if len(np.unique(dbscan_labels)) > 1:
                metrics.update({
                    'dbscan_silhouette': silhouette_score(
                        sample_features, 
                        dbscan_labels
                    ),
                    'dbscan_calinski': calinski_harabasz_score(
                        sample_features, 
                        dbscan_labels
                    )
                })
            
            return kmeans_labels, dbscan_labels, metrics
        except Exception as e:
            print(f"Error in clustering: {str(e)}")
            return None, None, {}
    
    def plot_results(self, pca_result, tsne_result, kmeans_labels, dbscan_labels):
        """Plot all analysis results with consistent sampling"""
        if any(x is None for x in [pca_result, tsne_result, kmeans_labels, dbscan_labels]):
            print("Skipping plotting due to missing results")
            return
            
        print("\nPlotting results...")
        try:
            fig, axes = plt.subplots(2, 2, figsize=(20, 20))
            
            # Use sampled data consistently
            if hasattr(self, 'is_sampled') and self.is_sampled:
                plot_features = pca_result[self.sample_indices]
                plot_labels = self.labels[self.sample_indices]
            else:
                plot_features = pca_result
                plot_labels = self.labels
            
            # Plot PCA with true labels
            self._scatter_plot(
                axes[0, 0], 
                plot_features, 
                plot_labels,
                'PCA Results with True Labels', 
                'PC1', 
                'PC2'
            )
            
            # Plot t-SNE with true labels
            self._scatter_plot(
                axes[0, 1], 
                tsne_result, 
                plot_labels,
                't-SNE Results with True Labels', 
                't-SNE1', 
                't-SNE2'
            )
            
            # Plot K-means clustering
            self._scatter_plot(
                axes[1, 0], 
                plot_features, 
                kmeans_labels,
                'PCA Results with K-means Clustering', 
                'PC1', 
                'PC2'
            )
            
            # Plot DBSCAN clustering
            self._scatter_plot(
                axes[1, 1], 
                plot_features, 
                dbscan_labels,
                'PCA Results with DBSCAN Clustering', 
                'PC1', 
                'PC2'
            )
            
            plt.tight_layout()
            plt.savefig('feature_analysis.png')
            plt.close()
        except Exception as e:
            print(f"Error in plotting: {str(e)}")
    
    def _scatter_plot(self, ax, data, labels, title, xlabel, ylabel):
        """Helper function to create scatter plots"""
        scatter = ax.scatter(data[:, 0], data[:, 1], c=labels, cmap='tab10')
        ax.set_title(title)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        plt.colorbar(scatter, ax=ax)
    
    def analyze_multiple_samples(self, n_samples=5, samples_per_class=200):
        """Analyze multiple random samples and create separate plots"""
        print(f"\nAnalyzing {n_samples} different samples...")
        
        unique_labels = np.unique(self.labels)
        n_classes = len(unique_labels)
        
        for sample_idx in range(n_samples):
            print(f"\nAnalyzing sample {sample_idx + 1}/{n_samples}")
            
            # Get balanced sample from each class
            sampled_indices = []
            for label in unique_labels:
                label_indices = np.where(self.labels == label)[0]
                if len(label_indices) > samples_per_class:
                    selected = np.random.choice(label_indices, samples_per_class, replace=False)
                    sampled_indices.extend(selected)
                else:
                    sampled_indices.extend(label_indices)
            
            # Convert to numpy array
            sampled_indices = np.array(sampled_indices)
            np.random.shuffle(sampled_indices)
            
            # Get sampled features and labels
            sampled_features = self.flattened_features[sampled_indices]
            sampled_labels = self.labels[sampled_indices]
            
            # Run analyses on sampled data
            # PCA
            pca = PCA(n_components=self.n_components)
            pca_result = pca.fit_transform(sampled_features)
            explained_variance = pca.explained_variance_ratio_
            
            # t-SNE
            tsne = TSNE(n_components=self.n_components, random_state=42)
            tsne_result = tsne.fit_transform(sampled_features)
            
            # Clustering
            kmeans = MiniBatchKMeans(n_clusters=n_classes, random_state=42)
            kmeans_labels = kmeans.fit_predict(sampled_features)
            
            dbscan = DBSCAN(eps=0.5, min_samples=5)
            dbscan_labels = dbscan.fit_predict(sampled_features)
            
            # Calculate metrics
            metrics = {
                'kmeans_silhouette': silhouette_score(sampled_features, kmeans_labels),
                'kmeans_calinski': calinski_harabasz_score(sampled_features, kmeans_labels)
            }
            
            if len(np.unique(dbscan_labels)) > 1:
                metrics.update({
                    'dbscan_silhouette': silhouette_score(sampled_features, dbscan_labels),
                    'dbscan_calinski': calinski_harabasz_score(sampled_features, dbscan_labels)
                })
            
            # Plot results
            fig, axes = plt.subplots(2, 2, figsize=(20, 20))
            fig.suptitle(f'Sample {sample_idx + 1} Analysis\n{samples_per_class} samples per class', 
                        fontsize=16)
            
            # Plot PCA with true labels
            self._scatter_plot(
                axes[0, 0],
                pca_result,
                sampled_labels,
                'PCA Results with True Labels',
                'PC1',
                'PC2'
            )
            
            # Plot t-SNE with true labels
            self._scatter_plot(
                axes[0, 1],
                tsne_result,
                sampled_labels,
                't-SNE Results with True Labels',
                't-SNE1',
                't-SNE2'
            )
            
            # Plot K-means clustering
            self._scatter_plot(
                axes[1, 0],
                pca_result,
                kmeans_labels,
                'PCA Results with K-means Clustering',
                'PC1',
                'PC2'
            )
            
            # Plot DBSCAN clustering
            self._scatter_plot(
                axes[1, 1],
                pca_result,
                dbscan_labels,
                'PCA Results with DBSCAN Clustering',
                'PC1',
                'PC2'
            )
            
            plt.tight_layout()
            plt.savefig(f'feature_analysis_sample_{sample_idx+1}.png')
            plt.close()
            
            # Print metrics for this sample
            print(f"\nSample {sample_idx + 1} Metrics:")
            print(f"PCA Explained Variance: {sum(explained_variance):.2%}")
            print("\nClustering Metrics:")
            for metric, value in metrics.items():
                print(f"{metric}: {value:.3f}")
        
        return True
    
    def analyze_features(self):
        """Run complete feature analysis with multiple samples"""
        # Preprocess features
        self.preprocess_features()
        
        # Run multiple sample analyses
        self.analyze_multiple_samples(n_samples=5, samples_per_class=200)
        
        # Run full dataset analysis for completeness
        pca_result, explained_variance = self.run_pca()
        tsne_result, sampled_labels = self.run_tsne()
        kmeans_labels, dbscan_labels, clustering_metrics = self.run_clustering()
        
        # Return results from full dataset analysis
        results = {
            'pca_result': pca_result,
            'tsne_result': tsne_result,
            'explained_variance': explained_variance if explained_variance is not None else np.array([0]),
            'clustering_metrics': clustering_metrics
        }
        
        return results

def run_feature_analysis(features, labels):
    """Main function to run feature analysis"""
    analyzer = FeatureAnalyzer(features, labels, batch_size=1000)
    results = analyzer.analyze_features()
    return results 

In [35]:
# Add feature analysis before training
print("\nRunning unsupervised feature analysis...")
analysis_results = run_feature_analysis(features, labels)
    
# Print feature analysis insights
print("\nFeature Analysis Insights:")
print(f"PCA Explained Variance: {analysis_results['explained_variance'].sum():.2%}")
print("\nClustering Metrics:")
for metric, value in analysis_results['clustering_metrics'].items():
    print(f"{metric}: {value:.3f}")
    
# Continue with model training if features look good
if analysis_results['explained_variance'].sum() > 0.7:  # 70% variance explained
    print("\nFeatures look good, proceeding with training...")


Running unsupervised feature analysis...
Preprocessing features...


Processing batches: 100%|██████████| 45/45 [00:10<00:00,  4.30it/s]


Preprocessed feature shape: (44025, 126)

Analyzing 5 different samples...

Analyzing sample 1/5


/home/21uec131/miniconda3/envs/crptonet/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)



Sample 1 Metrics:
PCA Explained Variance: 21.05%

Clustering Metrics:
kmeans_silhouette: 0.097
kmeans_calinski: 73.795
dbscan_silhouette: 0.015
dbscan_calinski: 5.157

Analyzing sample 2/5


...


Sample 4 Metrics:
PCA Explained Variance: 21.04%

Clustering Metrics:
kmeans_silhouette: 0.091
kmeans_calinski: 67.144

Analyzing sample 5/5


/home/21uec131/miniconda3/envs/crptonet/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)



Sample 5 Metrics:
PCA Explained Variance: 22.16%

Clustering Metrics:
kmeans_silhouette: 0.090
kmeans_calinski: 74.089
dbscan_silhouette: 0.012
dbscan_calinski: 5.187

Performing PCA analysis...

Performing t-SNE analysis...

Performing clustering analysis...


/home/21uec131/miniconda3/envs/crptonet/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)



Feature Analysis Insights:
PCA Explained Variance: 20.42%

Clustering Metrics:
kmeans_silhouette: 0.092
kmeans_calinski: 709.301
dbscan_silhouette: -0.207
dbscan_calinski: 15.862


In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        # Simple 1x1 conv for channel matching if needed
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Conv1d(in_channels, out_channels, kernel_size=1)
    
    def forward(self, x):
        residual = self.shortcut(x)
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        out += residual
        out = F.relu(out)
        return out

class SimpleNet(nn.Module):
    def __init__(self, input_channels=21, num_classes=5):
        super(SimpleNet, self).__init__()
        
        # Initial convolution with smaller number of filters
        self.conv1 = nn.Sequential(
            nn.Conv1d(input_channels, 16, kernel_size=3, padding=1),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        # Basic blocks with gradual channel increase
        self.block1 = BasicBlock(16, 32)
        self.block2 = BasicBlock(32, 64)
        self.block3 = BasicBlock(64, 128)
        
        # Global average pooling
        self.gap = nn.AdaptiveAvgPool1d(1)
        
        # Modified fully connected layers with increased regularization
        self.fc = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),  # Added batch normalization
            nn.Dropout(0.4),     # Increased dropout
            nn.Linear(64, num_classes)
        )
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, m):
        if isinstance(m, nn.Conv1d):
            nn.init.kaiming_uniform_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm1d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)
            nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # Input shape: (batch, seq_len, channels)
        x = x.permute(0, 2, 1)  # Change to (batch, channels, seq_len)
        
        # Apply layers with gradient checkpointing
        x = self.conv1(x)
        x = self.block1(x)
        x = F.max_pool1d(x, 2)
        x = self.block2(x)
        x = F.max_pool1d(x, 2)
        x = self.block3(x)
        
        # Global average pooling
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        
        # Classification
        x = self.fc(x)
        return x 

In [37]:
import torch
from torch.nn.utils import clip_grad_norm_
from tqdm import tqdm
import numpy as np

class EarlyStopping:
    def __init__(self, patience=7, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

def check_nan_inf(tensor, name=""):
    if torch.isnan(tensor).any() or torch.isinf(tensor).any():
        print(f"Found NaN or Inf in {name}")
        return True
    return False

def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=200):
    # Modified learning rate scheduler
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, 
        T_0=5,      # Reduced from 10
        T_mult=1,   # Reduced from 2
        eta_min=1e-5  # Increased from 1e-6
    )
    
    early_stopping = EarlyStopping(patience=15)  # Increased patience
    
    best_val_loss = float('inf')
    best_model_state = None
    best_epoch = 0
    best_accuracy = 0.0
    
    # Track metrics
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{epochs}')
        for batch_idx, (inputs, labels) in enumerate(pbar):
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            try:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                # Backward pass
                loss.backward()
                
                # Clip gradients
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                
                optimizer.step()
                
                # Update metrics
                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
                
                # Update progress bar
                current_lr = optimizer.param_groups[0]['lr']
                pbar.set_postfix({
                    'loss': f'{loss.item():.4f}',
                    'acc': f'{100 * train_correct/train_total:.2f}%',
                    'lr': f'{current_lr:.2e}'
                })
                
            except RuntimeError as e:
                print(f"Error in batch {batch_idx}: {str(e)}")
                continue
        
        # Calculate epoch metrics
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Calculate validation metrics
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        
        # Store metrics
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
        # Update learning rate
        scheduler.step()
        
        print(f'\nEpoch {epoch+1}/{epochs}:')
        print(f'Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')
        print(f'Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')
        print(f'Learning Rate: {optimizer.param_groups[0]["lr"]:.2e}')
        
        # Save best model
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            best_epoch = epoch
            print(f'New best model saved! Accuracy: {best_accuracy:.2f}%')
        
        # Early stopping
        early_stopping(avg_val_loss)
        if early_stopping.early_stop:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break
    
    print(f"\nTraining completed!")
    print(f"Best validation accuracy: {best_accuracy:.2f}% at epoch {best_epoch+1}")
    
    # Load best model
    model.load_state_dict(best_model_state)
    
    return model, {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies,
        'best_accuracy': best_accuracy,
        'best_epoch': best_epoch
    } 

In [38]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

class ModelEvaluator:
    def __init__(self, model, criterion, device):
        self.model = model
        self.criterion = criterion
        self.device = device
        self.classes = ['3DES', 'AES', 'ElGamal', 'KASUMI', 'RSA']
    
    def evaluate(self, test_loader):
        """Comprehensive model evaluation"""
        self.model.eval()
        test_loss = 0
        correct = 0
        total = 0
        
        # For detailed analysis
        all_preds = []
        all_labels = []
        all_probs = []
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                
                # Calculate probabilities
                probs = torch.nn.functional.softmax(outputs, dim=1)
                
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                # Store predictions and labels
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())
        
        # Calculate metrics
        accuracy = 100 * correct / total
        avg_loss = test_loss / len(test_loader)
        
        # Convert to numpy arrays
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)
        
        # Calculate per-class metrics
        results = self._calculate_metrics(all_labels, all_preds, all_probs)
        
        # Generate visualizations
        self._generate_visualizations(all_labels, all_preds, all_probs)
        
        return results
    
    def _calculate_metrics(self, true_labels, predictions, probabilities):
        """Calculate detailed metrics"""
        # Overall metrics
        accuracy = 100 * np.mean(predictions == true_labels)
        
        # Per-class accuracy
        class_correct = {}
        class_total = {}
        for i in range(len(self.classes)):
            idx = (true_labels == i)
            class_correct[self.classes[i]] = np.sum(predictions[idx] == i)
            class_total[self.classes[i]] = np.sum(idx)
        
        # Classification report
        report = classification_report(true_labels, predictions, 
                                    target_names=self.classes, 
                                    output_dict=True)
        
        # Confidence analysis
        confidences = np.max(probabilities, axis=1)
        avg_confidence = np.mean(confidences)
        
        return {
            'accuracy': accuracy,
            'class_correct': class_correct,
            'class_total': class_total,
            'classification_report': report,
            'avg_confidence': avg_confidence
        }
    
    def _generate_visualizations(self, true_labels, predictions, probabilities):
        """Generate evaluation visualizations"""
        # 1. Confusion Matrix
        cm = confusion_matrix(true_labels, predictions)
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=self.classes, yticklabels=self.classes)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.savefig('confusion_matrix.png')
        plt.close()
        
        # 2. Confidence Distribution
        plt.figure(figsize=(10, 6))
        for i in range(len(self.classes)):
            mask = true_labels == i
            plt.hist(np.max(probabilities[mask], axis=1), 
                    alpha=0.5, label=self.classes[i], bins=20)
        plt.title('Prediction Confidence Distribution')
        plt.xlabel('Confidence')
        plt.ylabel('Count')
        plt.legend()
        plt.savefig('confidence_distribution.png')
        plt.close()
        
        # 3. Error Analysis
        errors = predictions != true_labels
        error_probs = probabilities[errors]
        error_true = true_labels[errors]
        error_pred = predictions[errors]
        
        if len(error_probs) > 0:
            plt.figure(figsize=(10, 6))
            plt.scatter(error_true, np.max(error_probs, axis=1), 
                       alpha=0.5, c=error_pred, cmap='tab10')
            plt.title('Error Analysis')
            plt.xlabel('True Class')
            plt.ylabel('Prediction Confidence')
            plt.colorbar(label='Predicted Class')
            plt.savefig('error_analysis.png')
            plt.close()

def print_evaluation_results(results):
    """Print formatted evaluation results"""
    print("\nTest Set Evaluation Results:")
    print("=" * 50)
    
    print(f"\nOverall Accuracy: {results['accuracy']:.2f}%")
    print(f"Average Confidence: {results['avg_confidence']:.2f}")
    
    print("\nPer-class Accuracy:")
    print("-" * 30)
    for class_name in results['class_correct'].keys():
        correct = results['class_correct'][class_name]
        total = results['class_total'][class_name]
        print(f"{class_name}: {100 * correct / total:.2f}% ({correct}/{total})")
    
    print("\nDetailed Classification Report:")
    print("-" * 30)
    report = results['classification_report']
    for class_name in report.keys():
        if class_name in ['accuracy', 'macro avg', 'weighted avg']:
            continue
        metrics = report[class_name]
        print(f"\n{class_name}:")
        print(f"  Precision: {metrics['precision']:.3f}")
        print(f"  Recall: {metrics['recall']:.3f}")
        print(f"  F1-score: {metrics['f1-score']:.3f}")
        print(f"  Support: {metrics['support']}") 

In [39]:
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim


def main():
   
    # Run feature analysis
    print("\nRunning unsupervised feature analysis...")
    analysis_results = run_feature_analysis(features, labels)
    
    # Create dataset
    dataset = CryptoDataset(features, labels)
    
    # Split dataset
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    
    # Create train, validation, and test datasets
    generator = torch.Generator().manual_seed(42)  # For reproducibility
    train_dataset, val_dataset, test_dataset = random_split(
        dataset, 
        [train_size, val_size, test_size],
        generator=generator
    )
    
    # Initialize model and move to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SimpleNet(input_channels=features[0].shape[1], num_classes=5)
    
    # Multi-GPU support
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs!")
        model = nn.DataParallel(model)
    
    model = model.to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(
        model.parameters(),
        lr=0.0001,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        eps=1e-8
    )
    
    # Create dataloaders
    batch_size = 64
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=8,
        collate_fn=collate_fn,
        pin_memory=True,
        persistent_workers=True
    )
    
    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size,
        num_workers=8,
        collate_fn=collate_fn,
        pin_memory=True,
        persistent_workers=True
    )
    
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size,
        num_workers=8,
        collate_fn=collate_fn,
        pin_memory=True,
        persistent_workers=True
    )
    
    # Train the model
    print("\nStarting model training...")
    model, metrics = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epochs=200
    )
    
    # Evaluate on test set
    print("\nEvaluating on test set...")
    evaluator = ModelEvaluator(model, criterion, device)
    evaluation_results = evaluator.evaluate(test_loader)
    
    # Print evaluation results
    print_evaluation_results(evaluation_results)
    
    return model, metrics, analysis_results, evaluation_results

if __name__ == "__main__":
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.benchmark = True
    
    try:
        # Run main training and evaluation
        model, metrics, analysis_results, evaluation_results = main()
        
        # Results are now printed by print_evaluation_results
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

     # Print final results
    print("\nFinal Results:")
    print(f"Best Validation Accuracy: {metrics['best_accuracy']:.2f}%")
    print(f"Best Epoch: {metrics['best_epoch'] + 1}")
    print("\nFeature Analysis Results:")
    print(f"PCA Explained Variance: {analysis_results['explained_variance'].sum():.2%}")
    for metric, value in analysis_results['clustering_metrics'].items():
        print(f"{metric}: {value:.3f}")


Running unsupervised feature analysis...
Preprocessing features...


Processing batches: 100%|██████████| 45/45 [00:11<00:00,  3.85it/s]


Preprocessed feature shape: (44025, 126)

Analyzing 5 different samples...

Analyzing sample 1/5


/home/21uec131/miniconda3/envs/crptonet/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1930: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=3)



Sample 1 Metrics:
PCA Explained Variance: 21.38%

Clustering Metrics:
kmeans_silhouette: 0.088
kmeans_calinski: 71.684
dbscan_silhouette: -0.105
dbscan_calinski: 5.157

Analyzing sample 2/5


...


Epoch 97/200:
Train Loss: 0.6712, Train Accuracy: 90.39%
Val Loss: 0.7341, Val Accuracy: 88.57%
Learning Rate: 6.89e-05


Epoch 98/200: 100%|██████████| 482/482 [00:29<00:00, 16.40it/s, loss=0.5917, acc=90.88%, lr=6.89e-05]



Epoch 98/200:
Train Loss: 0.6603, Train Accuracy: 90.88%
Val Loss: 0.7245, Val Accuracy: 89.17%
Learning Rate: 4.11e-05
New best model saved! Accuracy: 89.17%
Early stopping triggered at epoch 98

Training completed!
Best validation accuracy: 89.17% at epoch 98

Evaluating on test set...


/tmp/ipykernel_24393/889549497.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.stack([torch.tensor(l) for l in labels])
/tmp/ipykernel_24393/889549497.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.stack([torch.tensor(l) for l in labels])
/tmp/ipykernel_24393/889549497.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.stack([torch.tensor(l) for l in labels])
/tmp/ipykernel_24393/889549497.py:96: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.cl


Test Set Evaluation Results:

Overall Accuracy: 88.60%
Average Confidence: 0.83

Per-class Accuracy:
------------------------------
3DES: 90.07% (1206/1339)
AES: 84.65% (1114/1316)
ElGamal: 91.31% (1219/1335)
KASUMI: 88.27% (1181/1338)
RSA: 88.65% (1132/1277)

Detailed Classification Report:
------------------------------

3DES:
  Precision: 0.941
  Recall: 0.901
  F1-score: 0.921
  Support: 1339.0

AES:
  Precision: 0.965
  Recall: 0.847
  F1-score: 0.902
  Support: 1316.0

ElGamal:
  Precision: 0.958
  Recall: 0.913
  F1-score: 0.935
  Support: 1335.0

KASUMI:
  Precision: 0.980
  Recall: 0.883
  F1-score: 0.929
  Support: 1338.0

RSA:
  Precision: 0.669
  Recall: 0.886
  F1-score: 0.763
  Support: 1277.0

Final Results:
Best Validation Accuracy: 89.17%
Best Epoch: 98

Feature Analysis Results:
PCA Explained Variance: 20.42%
kmeans_silhouette: 0.094
kmeans_calinski: 698.699
dbscan_silhouette: -0.192
dbscan_calinski: 17.082
